In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_set_util = DataSetUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/air/air_quality_beijing.csv', sep=',')
df = df.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
df = df.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
df = df.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
df = df.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
df = df.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
df = df.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
df = df.drop(labels=['No.6','day.6','year.6','month.6','hour.6','wd.6','station.6'], axis=1)
df = df.drop(labels=['No.7','day.7','year.7','month.7','hour.7','wd.7','station.7'], axis=1)
df = df.drop(labels=['No.8','day.8','year.8','month.8','hour.8','wd.8','station.8'], axis=1)
df = df.drop(labels=['No.9','day.9','year.9','month.9','hour.9','wd.9','station.9'], axis=1)
df = df.drop(labels=['No.10','day.10','year.10','month.10','hour.10','wd.10','station.10'], axis=1)
df = df.drop(labels=['No.11','day.11','year.11','month.11','hour.11','wd.11','station.11'], axis=1)
df.dropna(inplace=True)
data = data_set_util.clean_dataset(df)
data.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.11,SO2.11,NO2.11,CO.11,O3.11,TEMP.11,PRES.11,DEWP.11,RAIN.11,WSPM.11
23,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,15.0,14.0,41.0,500.0,31.0,1.0,1029.6,-18.4,0.0,1.8
24,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,10.0,11.0,35.0,400.0,39.0,0.5,1030.2,-18.3,0.0,1.4
25,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,8.0,9.0,33.0,400.0,40.0,-0.3,1030.2,-17.5,0.0,1.0
32,3.0,7.0,21.0,49.0,500.0,43.0,-0.4,1029.6,-17.6,0.0,...,7.0,17.0,53.0,500.0,17.0,0.0,1028.8,-16.8,0.0,1.3
33,10.0,14.0,47.0,62.0,700.0,29.0,0.6,1029.7,-16.7,0.0,...,11.0,17.0,50.0,500.0,23.0,1.1,1028.9,-16.8,0.0,1.0


In [5]:
data.describe()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.11,SO2.11,NO2.11,CO.11,O3.11,TEMP.11,PRES.11,DEWP.11,RAIN.11,WSPM.11
count,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,1.711700e+04,17117.000000,17117.000000,...,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,17117.000000,1.711700e+04,17117.000000,17117.000000,17117.000000
mean,83.296273,110.694170,15.999007,60.439440,1333.241865,63.306736,9.459692,1.072132e+03,1.863364,0.056780,...,112.914763,15.840153,56.153567,1378.364725,59.617327,9.631463,1.071910e+03,1.547029,0.060221,1.757043
std,82.384679,94.281669,20.084888,38.192672,1302.951080,740.002232,706.739740,7.728333e+03,13.731930,0.681897,...,95.566311,20.650638,35.882851,1227.163859,640.476953,706.740278,7.728334e+03,13.666349,0.722386,1.194589
min,3.000000,3.000000,1.000000,2.000000,100.000000,1.000000,-9575.000000,9.878000e+02,-35.300000,0.000000,...,3.000000,1.000000,2.000000,100.000000,1.000000,-9575.000000,9.878000e+02,-35.300000,0.000000,0.000000
25%,21.000000,37.000000,3.000000,31.000000,500.000000,8.000000,2.400000,1.004500e+03,-9.200000,0.000000,...,40.000000,2.000000,29.000000,600.000000,8.000000,2.600000,1.004100e+03,-9.400000,0.000000,0.900000
50%,58.000000,86.000000,8.000000,54.000000,900.000000,43.000000,12.600000,1.012500e+03,1.400000,0.000000,...,90.000000,8.000000,49.000000,1000.000000,42.000000,12.800000,1.012100e+03,1.000000,0.000000,1.400000
75%,116.000000,158.000000,20.000000,84.000000,1600.000000,80.000000,22.700000,1.021500e+03,14.500000,0.000000,...,155.000000,21.000000,77.000000,1700.000000,79.000000,22.900000,1.021600e+03,14.200000,0.000000,2.300000
max,713.000000,948.000000,185.000000,290.000000,10000.000000,78183.000000,21475.000000,1.012125e+06,28.500000,29.300000,...,811.000000,272.000000,235.000000,9800.000000,83538.000000,21475.000000,1.012125e+06,28.000000,29.300000,13.200000


In [6]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,PM2.5(t-1),PM10(t-1),SO2(t-1),NO2(t-1),CO(t-1),O3(t-1),TEMP(t-1),PRES(t-1),DEWP(t-1),RAIN(t-1),...,PM10.11(t),SO2.11(t),NO2.11(t),CO.11(t),O3.11(t),TEMP.11(t),PRES.11(t),DEWP.11(t),RAIN.11(t),WSPM.11(t)
24,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,10.0,11.0,35.0,400.0,39.0,0.5,1030.2,-18.3,0.0,1.4
25,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,8.0,9.0,33.0,400.0,40.0,-0.3,1030.2,-17.5,0.0,1.0
32,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,7.0,17.0,53.0,500.0,17.0,0.0,1028.8,-16.8,0.0,1.3
33,3.0,7.0,21.0,49.0,500.0,43.0,-0.4,1029.6,-17.6,0.0,...,11.0,17.0,50.0,500.0,23.0,1.1,1028.9,-16.8,0.0,1.0
34,10.0,14.0,47.0,62.0,700.0,29.0,0.6,1029.7,-16.7,0.0,...,14.0,22.0,52.0,600.0,20.0,1.9,1028.4,-17.1,0.0,2.0


In [7]:
df_forecats_columns = data.loc[:,'PM2.5':'WSPM.11'].columns

In [8]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'PM2.5(t-1)':'WSPM.11(t-1)']
            df_test = ttest.loc[:,'PM2.5(t-1)':'WSPM.11(t-1)']
            df_original = ttest.loc[:,'PM2.5(t)':'WSPM.11(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            df_original = pd.DataFrame(df_original.values,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [13]:
num_components_pca = 10
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [14]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 570
--------------------
training window 1140
--------------------
training window 1710
--------------------
training window 2280
--------------------
training window 2850
--------------------
training window 3420
--------------------
training window 3990
--------------------
training window 4560
--------------------
training window 5130


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 5700


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 6270
--------------------
training window 6840
--------------------
training window 7410
--------------------
training window 7980
--------------------
training window 8550
--------------------
training window 9120
--------------------
training window 9690
--------------------
training window 10260
--------------------
training window 10830
--------------------
training window 11400
--------------------
training window 11970


...

--------------------
training window 12540
--------------------
training window 13110
--------------------
training window 13680
--------------------
training window 14250
--------------------
training window 14820
--------------------
training window 15390


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 15960


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 16530


In [15]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [16]:
print("Statistics MIMO KPCA-NSFTS(test): ")
pd.set_option('display.max_rows', None)
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,PM2.5,22.189,12.689,34.183,0.820,13.361,0.094
1,PM10,32.413,19.815,32.159,0.697,13.456,0.114
2,SO2,5.317,3.129,27.463,0.761,11.561,0.098
3,NO2,14.397,9.484,26.622,0.705,10.576,0.121
4,CO,390.004,226.922,25.421,0.728,10.249,0.112
5,O3,400.001,76.991,132.674,0.673,21.200,0.131
6,TEMP,228.657,108.312,inf,0.727,17.905,0.108
7,PRES,4001.809,484.921,24.499,0.850,0.622,0.065
8,DEWP,1.605,0.978,inf,0.868,8.572,0.086
9,RAIN,0.283,0.064,NaN,-0.075,NaN,NaN


In [17]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_all.csv', index = False, header=True)

In [ ]:
#final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_all_2.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [9]:
num_components_pca = 20
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [10]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 570
--------------------
training window 1140


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 1710


...

--------------------
training window 12540
--------------------
training window 13110
--------------------
training window 13680
--------------------
training window 14250
--------------------
training window 14820
--------------------
training window 15390


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 15960


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 16530


In [11]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,PM2.5,23.993,16.460,53.356,0.801,18.955,0.102
1,PM10,34.259,23.898,47.510,0.684,17.782,0.120
2,SO2,6.674,4.582,52.765,0.666,19.841,0.118
3,NO2,15.998,11.552,33.692,0.647,13.072,0.133
4,CO,429.016,301.611,38.072,0.680,15.112,0.123
...,...,...,...,...,...,...,...
127,TEMP.11,248.856,162.359,inf,0.612,21.117,0.131
128,PRES.11,4342.395,1676.360,141.529,0.739,3.203,0.101
129,DEWP.11,1.971,1.400,inf,0.820,11.197,0.101
130,RAIN.11,0.307,0.152,NaN,0.234,NaN,NaN


In [12]:
pd.set_option('display.max_rows', None)
final_measures_pca

,variable,rmse,mae,mape,r2,smape,nrmse
0,PM2.5,23.993,16.460,53.356,0.801,18.955,0.102
1,PM10,34.259,23.898,47.510,0.684,17.782,0.120
2,SO2,6.674,4.582,52.765,0.666,19.841,0.118
3,NO2,15.998,11.552,33.692,0.647,13.072,0.133
4,CO,429.016,301.611,38.072,0.680,15.112,0.123
5,O3,334.962,146.576,1391.067,0.641,28.415,0.143
6,TEMP,248.855,162.360,inf,0.622,20.835,0.130
7,PRES,4342.439,1676.379,141.531,0.731,3.204,0.103
8,DEWP,1.949,1.373,inf,0.819,11.359,0.103
9,RAIN,0.285,0.138,inf,-0.132,98.618,NaN


In [21]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_all.csv', index = False, header=True)

In [ ]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_all_2.csv', index = False, header=True)

In [22]:
pd.set_option('display.max_rows', None)
final_measures_pca

,variable,rmse,mae,mape,r2,smape,nrmse
0,PM2.5,23.928,16.327,52.958,0.806,18.848,0.101
1,PM10,33.907,23.695,46.765,0.696,17.234,0.119
2,SO2,6.850,4.743,53.886,0.653,19.943,0.120
3,NO2,16.128,11.876,34.698,0.646,13.238,0.134
4,CO,428.567,299.791,36.809,0.690,14.625,0.122
5,O3,279.823,96.620,567.795,0.671,28.166,0.139
6,TEMP,247.689,162.084,inf,0.615,21.057,0.132
7,PRES,4344.116,1672.988,141.158,0.730,3.201,0.103
8,DEWP,1.955,1.391,inf,0.819,11.496,0.103
9,RAIN,0.275,0.135,inf,-0.078,98.566,NaN
